In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

class PharmaExogenousForecaster:
    def __init__(self):
        self.var_model = None
        self.xgb_models = {}
        self.feature_cols = []
        
    def create_time_features(self, df, date_col):
        """Create comprehensive time-based features"""
        df = df.copy()
        df[date_col] = pd.to_datetime(df[date_col])
        df = df.set_index(date_col)
        
        # Basic time features
        df['month'] = df.index.month
        df['quarter'] = df.index.quarter
        df['week'] = df.index.isocalendar().week
        df['day_of_week'] = df.index.dayofweek
        df['day_of_month'] = df.index.day
        df['year'] = df.index.year
        
        # Cyclical encoding for seasonality
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
        df['week_sin'] = np.sin(2 * np.pi * df['week'] / 52)
        df['week_cos'] = np.cos(2 * np.pi * df['week'] / 52)
        
        # Pharma-specific features
        df['is_month_end'] = df.index.is_month_end.astype(int)
        df['is_quarter_end'] = df.index.is_quarter_end.astype(int)
        df['is_year_end'] = df.index.is_year_end.astype(int)
        
        return df
    
    def create_lag_features(self, df, target_cols, lags=[1, 2, 3, 4, 12]):
        """Create lag features for target variables"""
        df = df.copy()
        
        for col in target_cols:
            if col in df.columns:
                for lag in lags:
                    df[f'{col}_lag_{lag}'] = df[col].shift(lag)
                
                # Rolling features
                df[f'{col}_roll_3'] = df[col].rolling(3).mean()
                df[f'{col}_roll_6'] = df[col].rolling(6).mean()
                df[f'{col}_roll_12'] = df[col].rolling(12).mean()
                
                # Trend features
                df[f'{col}_pct_change'] = df[col].pct_change()
                df[f'{col}_diff'] = df[col].diff()
        
        return df
    
    def check_stationarity(self, series):
        """Check if series is stationary using ADF test"""
        result = adfuller(series.dropna())
        return result[1] < 0.05  # p-value < 0.05 means stationary
    
    def prepare_var_data(self, df, target_cols):
        """Prepare data for VAR model"""
        var_data = df[target_cols].copy()
        
        # Check stationarity and difference if needed
        for col in target_cols:
            if not self.check_stationarity(var_data[col]):
                var_data[f'{col}_diff'] = var_data[col].diff()
                print(f"Applied differencing to {col}")
        
        return var_data.dropna()
    
    def fit_var_model(self, df, target_cols, maxlags=12):
        """Fit VAR model for multivariate forecasting"""
        var_data = self.prepare_var_data(df, target_cols)
        
        # Fit VAR model
        self.var_model = VAR(var_data)
        var_fit = self.var_model.fit(maxlags=maxlags, ic='aic')
        
        print(f"VAR model fitted with {var_fit.k_ar} lags")
        print(f"AIC: {var_fit.aic}")
        
        return var_fit
    
    def fit_xgb_models(self, df, target_cols, test_size=0.2):
        """Fit XGBoost models for each target variable"""
        
        # Create features
        df_features = self.create_time_features(df, df.index.name or 'date')
        df_features = self.create_lag_features(df_features, target_cols)
        
        # Remove target columns from features
        feature_cols = [col for col in df_features.columns if col not in target_cols]
        feature_cols = [col for col in feature_cols if not col.endswith('_diff')]
        self.feature_cols = feature_cols
        
        # Split data
        split_idx = int(len(df_features) * (1 - test_size))
        train_data = df_features.iloc[:split_idx]
        test_data = df_features.iloc[split_idx:]
        
        results = {}
        
        for target_col in target_cols:
            print(f"\nTraining XGBoost for {target_col}...")
            
            # Prepare training data
            X_train = train_data[feature_cols].fillna(0)
            y_train = train_data[target_col].fillna(0)
            X_test = test_data[feature_cols].fillna(0)
            y_test = test_data[target_col].fillna(0)
            
            # Train XGBoost
            xgb_model = xgb.XGBRegressor(
                n_estimators=100,
                max_depth=6,
                learning_rate=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42
            )
            
            xgb_model.fit(X_train, y_train)
            self.xgb_models[target_col] = xgb_model
            
            # Evaluate
            y_pred = xgb_model.predict(X_test)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            
            results[target_col] = {
                'MAE': mae,
                'RMSE': rmse,
                'Feature_Importance': dict(zip(feature_cols, xgb_model.feature_importances_))
            }
            
            print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}")
        
        return results
    
    def forecast_xgb(self, df, target_cols, steps_ahead=12):
        """Generate forecasts using XGBoost models"""
        
        # Prepare last known data
        df_features = self.create_time_features(df, df.index.name or 'date')
        df_features = self.create_lag_features(df_features, target_cols)
        
        forecasts = {}
        last_data = df_features.iloc[-1:].copy()
        
        for step in range(steps_ahead):
            step_forecasts = {}
            
            for target_col in target_cols:
                if target_col in self.xgb_models:
                    # Get features for prediction
                    X_pred = last_data[self.feature_cols].fillna(0)
                    pred = self.xgb_models[target_col].predict(X_pred)[0]
                    step_forecasts[target_col] = pred
            
            # Store forecasts
            for target_col, pred in step_forecasts.items():
                if target_col not in forecasts:
                    forecasts[target_col] = []
                forecasts[target_col].append(pred)
            
            # Update last_data with predictions for next step
            # This is a simplified approach - you might want more sophisticated updating
            for target_col, pred in step_forecasts.items():
                last_data[target_col] = pred
        
        return forecasts
    
    def forecast_var(self, steps_ahead=12):
        """Generate forecasts using VAR model"""
        if self.var_model is None:
            raise ValueError("VAR model not fitted yet")
        
        var_fit = self.var_model.fit()
        forecast = var_fit.forecast(var_fit.y, steps=steps_ahead)
        
        return forecast
    
    def hybrid_forecast(self, df, target_cols, steps_ahead=12):
        """Combine VAR and XGBoost forecasts"""
        
        # Get VAR forecasts
        try:
            var_forecasts = self.forecast_var(steps_ahead)
            var_available = True
        except:
            var_available = False
            print("VAR forecasting failed, using XGBoost only")
        
        # Get XGBoost forecasts
        xgb_forecasts = self.forecast_xgb(df, target_cols, steps_ahead)
        
        # Combine forecasts (weighted average)
        if var_available:
            combined_forecasts = {}
            for i, target_col in enumerate(target_cols):
                if target_col in xgb_forecasts:
                    # Simple average (you can use more sophisticated weighting)
                    var_vals = var_forecasts[:, i]
                    xgb_vals = xgb_forecasts[target_col]
                    combined_forecasts[target_col] = [
                        (v + x) / 2 for v, x in zip(var_vals, xgb_vals)
                    ]
        else:
            combined_forecasts = xgb_forecasts
        
        return combined_forecasts

# Example usage
def example_usage():
    """Example of how to use the forecaster"""
    
    # Create sample pharma data
    dates = pd.date_range('2020-01-01', '2024-12-31', freq='M')
    np.random.seed(42)
    
    # Simulate correlated pharma data
    n = len(dates)
    calls_data = 1000 + 200 * np.sin(2 * np.pi * np.arange(n) / 12) + np.random.normal(0, 50, n)
    email_data = 800 + 150 * np.sin(2 * np.pi * np.arange(n) / 12 + np.pi/4) + np.random.normal(0, 40, n)
    sales_data = 5000 + 0.3 * calls_data + 0.4 * email_data + np.random.normal(0, 100, n)
    
    df = pd.DataFrame({
        'date': dates,
        'calls': calls_data,
        'emails': email_data,
        'sales': sales_data
    })
    
    # Initialize forecaster
    forecaster = PharmaExogenousForecaster()
    target_cols = ['calls', 'emails', 'sales']
    
    # Fit models
    print("Fitting XGBoost models...")
    xgb_results = forecaster.fit_xgb_models(df.set_index('date'), target_cols)
    
    print("\nFitting VAR model...")
    try:
        var_results = forecaster.fit_var_model(df.set_index('date'), target_cols)
    except Exception as e:
        print(f"VAR fitting failed: {e}")
    
    # Generate forecasts
    print("\nGenerating hybrid forecasts...")
    forecasts = forecaster.hybrid_forecast(df.set_index('date'), target_cols, steps_ahead=6)
    
    # Display results
    print("\nForecasts for next 6 months:")
    for target_col, values in forecasts.items():
        print(f"{target_col}: {[round(v, 2) for v in values]}")
    
    return forecaster, forecasts

# Run example
if __name__ == "__main__":
    forecaster, forecasts = example_usage()